# ERK-KTR Full FOV Stimulation Pipeline
This notebook showcases how to use the ERK-KTR full FOV stimulation pipeline. The pipeline is designed to simulate the full field of view (FOV) stimulation of a cells with the ERK-KTR biosensor. This demo is adapted to run on our TI-Eclipse microscope called Jungfrau (see our [Mic config repository](https://github.com/pertzlab/pertzlab_mic_configs) for the full configuration file). 

### Import required libraries

In [ ]:
import os
import time

os.environ["QT_LOGGING_RULES"] = (
    "*.debug=false; *.warning=false"  # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook
)
os.environ["MICROMANAGER_PATH"] = "C:\\Program Files\\Micro-Manager-2.0"

from rtm_pymmcore.data_structures import Fov, Channel, StimTreatment
from pprint import pprint
import pandas as pd
import numpy as np
import dataclasses
import random
import pymmcore_plus

mmc = pymmcore_plus.CMMCorePlus()

### Experimental Settings

In [ ]:
## Configuration options
N_FRAMES = 60
FIRST_FRAME_STIMULATION = 10

SLEEP_BEFORE_EXPERIMENT_START_in_H = 0
USE_AUTOFOCUS_EVENT = False

TIME_BETWEEN_TIMESTEPS = 60  # time in seconds between frames
TIME_PER_FOV = 6  # time in seconds per fov


## Storage path for the experiment
base_path = "C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp"
experiment_name = "exp_test"
path = os.path.join(base_path, experiment_name)

# Define Channels for which Images are taken. If no power is defined, the default power of the device will be used,
# for example, see the second channel "Cy5" below. The default power is set in the GUI
channels = []
channels.append(Channel(name="miRFP", exposure=150))
channels.append(Channel(name="mScarlet3", exposure=100))


# Condition mapping to FOVs. This is used to create a dataframe with the conditions and the FOVs.
condition = [
    "FGFR_high",
    "TrkA_high",
]  # Example of adding a condition to the dataframe. Stimulation will be repeated for each condition.
# condition = ["optoFGFR_high"] * 24 + ["optoFGFR"] * 24 # Example of adding multiple conditions to the dataframe. n repreats the amount of times the condition is repeated.

n_fovs_per_condition = 1  ## change this variable to the amount of fovs that you have per cell line. If only one cell line is set, this value will
# automatically set to total amount of fovs.

n_fovs_per_well = 6  ## change this variable to the amount of fovs that you have per well. Set to None if you are not working with wellplate.


# Stimulation parameters for optogenetics. The stimulation will be repeated for each condition.

stim_exposures = [
    60
]  # or e.g. [10, 20, 30] for different exposures. The exposure time is the time that the LED is on.
# Define the stimulation timesteps
stim_timesteps = [
    range(FIRST_FRAME_STIMULATION, N_FRAMES, 2)
]  # Using range to define timesteps from FIRST_FRAME_STIMULATION to N_FRAMES with step 2

# Combine the different paramters in stim_exposure and stim timestep to create stim_treatments which represents all possible combinations
stim_treatments = [
    StimTreatment(
        stim_channel_name="CyanStim",
        stim_channel_group="TTL_ERK",
        stim_timestep=stim_timestep,
        stim_exposure=stim_exposure,
        stim_power=3,
        stim_channel_device_name="Spectra",
        stim_channel_power_property_name="Cyan_Level",
    )
    for stim_exposure in stim_exposures
    for stim_timestep in stim_timesteps
]
for stim_treatment in stim_treatments:
    if isinstance(stim_treatment.stim_timestep, range):
        stim_treatment.stim_timestep = tuple(stim_treatment.stim_timestep)

## Define the Tools that you are using for the experiment
from rtm_pymmcore.segmentation.stardist import SegmentatorStardist
from rtm_pymmcore.stimulation.base_stimulation import StimWholeFOV
from rtm_pymmcore.tracking.trackpy import TrackerTrackpy
from rtm_pymmcore.feature_extraction.erk_ktr import FE_ErkKtr

segmentators = [
    {
        "name": "labels",
        "class": SegmentatorStardist(min_size=200, prob_thresh=0.55),
        "use_channel": 0,
        "save_tracked": True,
    },
]
stimulator = StimWholeFOV()
feature_extractor = FE_ErkKtr("labels")
tracker = TrackerTrackpy()

pprint(stim_treatments)


from rtm_pymmcore.img_processing_pip import ImageProcessingPipeline

pipeline = ImageProcessingPipeline(
    storage_path=path,
    segmentators=segmentators,
    feature_extractor=feature_extractor,
    tracker=tracker,
    stimulator=stimulator,
)

### Load device and set startup channel

In [ ]:
mmc.loadSystemConfiguration(
    "E:\\pertzlab_mic_configs\\micromanager\\\Jungfrau\\TiFluoroJungfrau_w_TTL_DIGITALIO.cfg"
)
mmc.setConfig(groupName="System", configName="Startup")
mmc.setChannelGroup(channelGroup="TTL_ERK")

### GUI - Napari Micromanager

#### Load GUI

In [ ]:
### Base GUI ###
from napari_micromanager import MainWindow
import napari

viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)
data_mda_fovs = None

### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget

mdawidget = MDAWidget(mmcore=mmc)
viewer.window.add_dock_widget(mdawidget)
load_from_file = False

#### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [ ]:
### Break connection
# mm_wdg._core_link.cleanup()

In [ ]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink

mm_wdg._core_link = CoreViewerLink(viewer, mmc)

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [ ]:
import json

file = os.path.join(path, "fovs.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)
load_from_file = True

### Use FOVs to generate dataframe for acquisition

In [ ]:
n_fovs_simultaneously = TIME_BETWEEN_TIMESTEPS // TIME_PER_FOV
timesteps = range(N_FRAMES)

start_time = 0
if not load_from_file:
    data_mda_fovs = mdawidget.value().stage_positions
    data_mda_fovs_dict = []
    for data_mda in data_mda_fovs:
        data_mda_fovs_dict.append(data_mda.model_dump())
    data_mda_fovs = data_mda_fovs_dict
    if data_mda_fovs is None:
        assert False, "No fovs selected. Please select fovs in the MDA widget"

dfs = []
fovs = []
for fov_index, fov in enumerate(data_mda_fovs):
    fov_object = Fov(fov_index)
    fovs.append(fov_object)
    fov_group = fov_index // n_fovs_simultaneously
    start_time = fov_group * TIME_BETWEEN_TIMESTEPS * len(timesteps)
    if len(condition) == 1:
        condition_fov = condition[0]
    else:
        condition_fov = condition[fov_index // n_fovs_per_condition]
    for timestep in timesteps:
        row = {
            "fov_object": fov_object,
            "fov": fov_index,
            "fov_x": fov.get("x"),
            "fov_y": fov.get("y"),
            "fov_z": fov.get("z"),
            "fov_name": str(fov_index) if fov["name"] is None else fov.name,
            "timestep": timestep,
            "time": start_time + timestep * TIME_BETWEEN_TIMESTEPS,
            "cell_line": condition_fov,
            "channels": tuple(dataclasses.asdict(channel) for channel in channels),
            "fname": f"{str(fov_index).zfill(3)}_{str(timestep).zfill(5)}",
        }
        dfs.append(row)

df_acquire = pd.DataFrame(dfs)

print(f"Total Experiment Time: {df_acquire['time'].max()/3600}h")

for stim_treatment in stim_treatments:
    if isinstance(stim_treatment.stim_timestep, range):
        stim_treatment.stim_timestep = tuple(stim_treatment.stim_timestep)

n_fovs = len(data_mda_fovs)
n_stim_treatments = len(stim_treatments)
if n_stim_treatments > 0:
    n_fovs_per_stim_condition = n_fovs // n_stim_treatments // len(np.unique(condition))
    stim_treatment_tot = []
    random.shuffle(stim_treatments)
    if n_fovs_per_well is not None:
        for stim_treat in stim_treatments:
            stim_treatment_tot.extend([stim_treat] * n_fovs_per_well)

    else:
        for fov_index in range(0, n_fovs_per_stim_condition):
            stim_treatment_tot.extend(stim_treatments)
        random.shuffle(stim_treatment_tot)

        if n_fovs % n_stim_treatments != 0:
            print(
                f"Warning: Not equal number of fovs per stim condition. {n_fovs % n_stim_treatments} fovs will have repeated treatment"
            )
            stim_treatment_tot.extend(stim_treatments[: n_fovs % n_stim_treatments])
    print(f"Doing {n_fovs_per_stim_condition} experiment per stim condition")

    if len(condition) == 1:
        n_fovs_per_condition = n_fovs
    else:
        stim_treatment_tot = stim_treatment_tot * len(np.unique(condition))

    df_acquire = pd.merge(
        df_acquire, pd.DataFrame(stim_treatment_tot), left_on="fov", right_index=True
    )

    # Add stim column that checks if current timestep is in the stim_timestep tuple
    df_acquire["stim"] = df_acquire.apply(
        lambda row: (
            row["timestep"] in row["stim_timestep"]
            if isinstance(row["stim_timestep"], tuple) and row["stim_exposure"] > 0
            else False
        ),
        axis=1,
    )

df_acquire = df_acquire.dropna(axis=1, how="all")
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", True)
df_acquire = df_acquire.sort_values(by=["time", "fov"]).reset_index(drop=True)
df_acquire

### Run experiment

In [ ]:
pymmcore_plus.configure_logging(stderr_level="WARNING")
for _ in range(0, SLEEP_BEFORE_EXPERIMENT_START_in_H * 3600):
    time.sleep(1)
from rtm_pymmcore.controller import Controller, Analyzer
from queue import Queue

try:
    mm_wdg._core_link.cleanup()
except:
    pass


analyzer = Analyzer(pipeline)
queue = Queue()
controller = Controller(
    analyzer,
    mmc,
    queue,
)
controller.run(df_acquire)
print("Experiment finished")

fovs_i_list = os.listdir(os.path.join(path, "tracks"))
fovs_i_list.sort()
dfs = []
for fov_i in fovs_i_list:
    track_file = os.path.join(path, "tracks", fov_i)
    df = pd.read_parquet(track_file)
    dfs.append(df)
pd.concat(dfs).to_parquet(os.path.join(path, "exp_data.parquet"))